# Finding Top Clips

In [20]:
import requests
import os
import shutil
import datetime

date_today = datetime.datetime.today().strftime('%Y-%m-%d')
clips_url = "https://api.twitch.tv/kraken/clips/top"

clip_params = {
    "game": "Overwatch",
    "period": "day",
    "limit": 10,
    "language": "en"
}

clip_headers = {
    "Accept": "application/vnd.twitchtv.v5+json",
    "Client-ID": "bmqi273g9qwdztef0giu91l7150o25"
}


In [21]:
#request clips
request = requests.get(clips_url, params=clip_params, headers=clip_headers)
clips = request.json()["clips"]

In [22]:
#parse appropriate information
tmpClips = []

for clip in clips:
    tmpClips.append({
        'broadcaster': clip['broadcaster']['display_name'],
        'created_at': datetime.datetime.strptime(clip['created_at'], "%Y-%m-%dT%H:%M:%SZ"),
        'url': clip['url'],
        'duplicate': False
        })
    print(clip['broadcaster']['display_name'])

#remove duplicates
nonDuplicateClips = []

for i in range(len(tmpClips)):
    for j in range(i + 1, len(tmpClips)):
        minsDiff = abs((tmpClips[i]['created_at'] - tmpClips[j]['created_at']).days * 24 * 60)
        sameStreamer = tmpClips[i]['broadcaster'] == tmpClips[j]['broadcaster']
        if sameStreamer and minsDiff < 30:
            tmpClips[i]['duplicate'] = True
            break
    if tmpClips[i]['duplicate'] == False:
        nonDuplicateClips.append(tmpClips[i])

m0xyy
supertf
dafran
xQcOW
dafran
xQcOW
m0xyy
Alinity
Necros
space


In [23]:
#cite broadcasters
clip_urls = [ clip["url"].split("?")[0] for clip in nonDuplicateClips ]
broadcaster_urls = [clip["broadcaster"]['channel_url'] for clip in clips]
broadcaster_urls_uniq = list(set(broadcaster_urls))

#Clear download folders, text file
folders = ['downloads', 'transcoded_outputs']
for folder in folders:
    if not os.path.isdir(folder):
        os.makedirs(folder)
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        if os.path.isfile(file_path):
            os.unlink(file_path)

#write file of clip urls to download
with open("clips.txt", "w") as clip_f:
    for url in clip_urls:
        clip_f.write(url + "\n")
        
#write description to video
with open('ytdescription.txt', 'w') as yt_f:
    yt_description = "Fortnite Stream Highlights\nCheck out the featured channels:\n"
    for url in broadcaster_urls_uniq:
        yt_description += url + "\n"
    yt_f.write(yt_description)

#write list of broadcasters
with open('broadcasters.txt', 'w') as broadcaster_f:
    for broadcaster_url in broadcaster_urls:
        broadcaster_f.write(broadcaster_url + "\n")

# Downloading Clips

In [24]:
import re
import urllib.request
import requests
import sys

basepath = 'downloads/'
base_clip_path = 'https://clips-media-assets2.twitch.tv/'

def retrieve_mp4_data(slug):
    cid = "bmqi273g9qwdztef0giu91l7150o25"#sys.argv[1]
    clip_info = requests.get(
        "https://api.twitch.tv/helix/clips?id=" + slug,
        headers={"Client-ID": cid}).json()
    thumb_url = clip_info['data'][0]['thumbnail_url']
    title = clip_info['data'][0]['title']
    slice_point = thumb_url.index("-preview-")
    mp4_url = thumb_url[:slice_point] + '.mp4'
    return mp4_url, title

def dl_progress(count, block_size, total_size):
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%" % percent)
    sys.stdout.flush()

In [25]:
# for each clip in clips.txt
with open('clips.txt', 'r') as read_clips:
    for clip in read_clips:
        slug = clip.split('/')[3].replace('\n', '')
        mp4_url, clip_title = retrieve_mp4_data(slug)
        regex = re.compile('[^a-zA-Z0-9_]')
        clip_title = clip_title.replace(' ', '_')
        out_filename = regex.sub('', clip_title) + '.mp4'
        output_path = (basepath + out_filename)

        print('\nDownloading clip slug: ' + slug)
        print('"' + clip_title + '" -> ' + out_filename)
        print(mp4_url)
        urllib.request.urlretrieve(mp4_url, output_path, reporthook=dl_progress)
        print('\nDone.')

print('Finished downloading all the videos.')


"super_pepega_brain" -> super_pepega_brain.mp4
https://clips-media-assets2.twitch.tv/AT-cm%7C380123036.mp4
...100%
Done.

"Dafran_5k_poggers" -> Dafran_5k_poggers.mp4
https://clips-media-assets2.twitch.tv/32200215616-offset-4974.mp4
...100%
Done.

"TRACC" -> TRACC.mp4
https://clips-media-assets2.twitch.tv/AT-cm%7C379954860.mp4
...100%
Done.

"xQc_walking_into_the_club" -> xQc_walking_into_the_club.mp4
https://clips-media-assets2.twitch.tv/AT-cm%7C380039692.mp4
...100%
Done.

"FeelsStrongMan" -> FeelsStrongMan.mp4
https://clips-media-assets2.twitch.tv/32198281648-offset-13664.mp4
...100%
Done.

"Alinity_<3_Kemal" -> Alinity_3_Kemal.mp4
https://clips-media-assets2.twitch.tv/32196587552-offset-11684.mp4
...100%
Done.

"gamer" -> gamer.mp4
https://clips-media-assets2.twitch.tv/AT-cm%7C380190404.mp4
...100%
Done.

"Space_with_an_insane_grav_eat" -> Space_with_an_insane_grav_eat.mp4
https://clips-media-assets2.twitch.tv/AT-cm%7C380174121.mp4
...100%
Done.
Finished downloading all the videos

# Stitch Clips Together

In [27]:
from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
import datetime

indir = 'downloads/'
outdir = 'final_outputs/'
INPUT_EXT1 = '.MP4'
clips = []

for infile in os.listdir(indir):
    if infile.upper().endswith(INPUT_EXT1):
        clips.append(VideoFileClip(indir+infile))
        
if not os.path.isdir(outdir):
        os.makedirs(outdir)
        
final_clip = concatenate_videoclips(clips)
final_clip.write_videofile(date_today+".mp4")

Imageio: 'ffmpeg-win32-v3.2.4.exe' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-win32-v3.2.4.exe (34.1 MB)
Downloading: 8192/35749888 bytes (0.0%)49152/35749888 bytes (0.1%)106496/35749888 bytes (0.3%)163840/35749888 bytes (0.5%)221184/35749888 bytes (0.6%)278528/35749888 bytes (0.8%)335872/35749888 bytes (0.9%)393216/35749888 bytes (1.1%)450560/35749888 bytes (1.3%)507904/35749888 bytes (1.4%)565248/35749888 bytes (1.6%)622592/35749888 bytes (1.7%)679936/35749888 bytes (1.9%)737280/35749888 bytes (2.1%)79462

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5249/5249 [00:08<00:00, 633.78it/s]


[MoviePy] Done.
[MoviePy] Writing video 2019-01-16.mp4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14283/14283 [45:26<00:00,  3.95it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: 2019-01-16.mp4 



# Stitch Clips Together

In [4]:
import os
import sys
import tempfile
import shutil
import logging
import datetime

APP_NAME = 'mp4-merge'
indir = 'downloads/'
outdir = 'final_outputs/'
INPUT_EXT1 = '.MP4'
OUT_EXT1 = '.mpg'
date = datetime.datetime.today().strftime('%Y-%m-%d')
MP4_RESULT_FILE = outdir + date + '-output.mp4'
VIDEO_QUALITY = 22 #22=10300 Kbps, 23=, 24=6000 Kbps

if not os.path.isdir(outdir):
        os.makedirs(outdir)

log = logging.getLogger(APP_NAME)
log.setLevel(logging.DEBUG)
lfh = logging.FileHandler(APP_NAME + '.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
lfh.setFormatter(formatter)
log.addHandler(lfh)

tmpdir = tempfile.mkdtemp()
log.info('Temp dir ' + tmpdir)

log.info('MP4s -> MPGs')
mpg_file_list = list()
for infile in os.listdir(indir):
    if infile.upper().endswith(INPUT_EXT1):
        outfile = tmpdir + '/' + infile + OUT_EXT1
        cmd = 'ffmpeg -i ' + indir + '/' + infile + ' -qscale:v 1 ' + outfile
        log.info(cmd)
        os.system(cmd)
        mpg_file_list.append(outfile)


log.info('MPGs -> MP4')
mpg_files_str = ''
for mpg_file in mpg_file_list:
    mpg_files_str = mpg_files_str + mpg_file + '|'

cmd = 'ffmpeg -i concat:"' + mpg_files_str + '" -c:v libx264 -preset slow -crf ' + str(VIDEO_QUALITY) + ' -c:a aac -strict -2 -vf select=concatdec_select -af aselect=concatdec_select,aresample=async=1 ' + MP4_RESULT_FILE
log.info(cmd)
os.system(cmd)


log.info('Removing temp dir ' + tmpdir + '...')
shutil.rmtree(tmpdir)
log.info('DONE')
lfh.close()